<a href="https://colab.research.google.com/github/adolfoguimaraes/datascience/blob/main/code/08_laboratorio_avaliacao_aprendizado_supervisionado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aprendizado Supervisionado

Vamos trabalhar com mais uma base e testar diferentes algortimos de Machine Learning. No entanto, nosso foco vai ser olhar as métricas de avaliação estudadas em sala de aula.

In [ ]:
import pandas as pd
from tabulate import tabulate
from sklearn.model_selection import cross_validate

import warnings
warnings.filterwarnings('ignore')


## Base dos Dados 

Vamos utilizar a base de dados disponível nesse link: https://archive.ics.uci.edu/ml/datasets/spambase. A base está disponível na pasta `datasets` com o nome `SpamDataset.csv`. A versão deste repositório já está com os nomes das colunas. No repositório original, essas informações estão em arquivos separados. Não foi feita nenhuma modificação nos dados, apenas a inclusão no nome das colunas. 

In [ ]:
data_ = pd.read_csv("../datasets/SpamDataset.csv")
data_

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,classe
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.000,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.010,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4596,0.31,0.00,0.62,0.0,0.00,0.31,0.00,0.00,0.00,0.00,...,0.000,0.232,0.0,0.000,0.000,0.000,1.142,3,88,0
4597,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.353,0.000,0.000,1.555,4,14,0
4598,0.30,0.00,0.30,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.102,0.718,0.0,0.000,0.000,0.000,1.404,6,118,0
4599,0.96,0.00,0.00,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.057,0.0,0.000,0.000,0.000,1.147,5,78,0


Antes de começarmos a definir nossa estratégia de treinamento, vamos olhar como estão distribuídos esses dados. 

In [ ]:
pd.DataFrame(data_['classe'].value_counts(normalize=True))

,classe
0,0.605955
1,0.394045


Isso indica que 60% das instâncias estão classificadas como não-spam e 40% como Spam. Existe um desbalanceamento dos dados e precisamos pensar em qual classe vamos levar em consideração na hora de treinar nossos modelos.

## Definindo o Treinamento

### Separação Dados e Classe

In [ ]:
X = data_[data_.columns[:-1]]
y = data_[data_.columns[-1]]

### Métodos utilizados

Vamos criar uma estratégia de treinamento de diferentes modelos de Machine Learning a tarefa de classificação de SPAM. Vamos testar os seguintes modelos e parâmetros:

* **KNN:** Variar o valor de `n_neighbors` de 1 a 15. 
* **SVM:** Usar o kernel rbf e variar os parâmetros `C` e `gamma` com os valores [0.01, 0.1, 1, 10, 100] (para o `C`) e [0.01, 0.1, 1] para o `gamma`.
* **Árvore de Decisão:** Será variado o `max_depth` com os valores [None, 1, 5, 10, 20, 30]. 
* **Regressão Logística:** Vamos fixar o solver em `liblinear` e variar `C` com os valores: [0.01, 0.1, 1, 10, 100].
* **Radom Forest:** Vamos varia o `n_estimators` e o `max_depth` com so valores [10, 50, 100, 200] (para o `n_estimators`) e [None, 1, 5, 10, 20, 30] (para o `max_depth`),

In [ ]:
# Instanciando os métodos utilizados e o método para aplicar GridSearch

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

### Parâmetros

Foram escolhidas apenas algumas variações de parâmetro com base no que foi discutido em sala.

In [ ]:
gridsearch_param_knn = {'n_neighbors': list(range(1,16))}
gridsearch_param_svm = {'C': [0.1, 1, 10], 'gamma': [0.01, 0.1, 1]}
gridsearch_param_dt = {'max_depth': [None, 1, 5, 10, 20, 30]}
gridsearch_param_lr = {'C': [0.01, 0.1, 1, 10, 100]}
gridsearch_param_rf =  {'n_estimators': [10, 50, 100, 200], 'max_depth': [None, 1, 5, 10, 20, 30]}


### Execução do Treinamento

In [ ]:

# Método que aplica o GridSearch com os parâmetros definido e retorna o melhor resultado de acordo com a métrica de interesse (score)

def fit_grid_search(model, params, score, X, y):

    gridsearch_model = GridSearchCV(model,param_grid=params, cv=5, scoring=['accuracy','precision','recall','f1'], refit=score)

    gridsearch_model.fit(X, y)


    print("Melhores parâmetros:", gridsearch_model.best_params_)
    print("Melhor score (%s): %f" % (score,gridsearch_model.best_score_ ))
    
    
    dt_frame = pd.DataFrame(gridsearch_model.cv_results_)

    dt_frame = dt_frame.query("rank_test_" + score + " == 1")
    dt_frame = dt_frame[['mean_fit_time','std_fit_time','mean_score_time','std_score_time','mean_test_accuracy','mean_test_precision','mean_test_recall','mean_test_f1']]

    return dt_frame


In [ ]:
# Instanciando os métodos com os parâmetros padrões ou fixos.
knn_ = KNeighborsClassifier()
svm_ = SVC(kernel='rbf')
dt_ = DecisionTreeClassifier()
lr_ = LogisticRegression(solver='liblinear')
rf_ = RandomForestClassifier()

#### KNN

In [ ]:
fit_grid_search(knn_, gridsearch_param_knn, 'accuracy', X, y)

Melhores parâmetros: {'n_neighbors': 1}
Melhor score (accuracy): 0.777007


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,mean_test_accuracy,mean_test_precision,mean_test_recall,mean_test_f1
0,0.006236,0.000861,0.135637,0.042277,0.777007,0.717829,0.724201,0.720072


#### Árvore de Decisão

In [ ]:
fit_grid_search(dt_, gridsearch_param_dt, 'accuracy', X, y)

Melhores parâmetros: {'max_depth': 10}
Melhor score (accuracy): 0.900888


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,mean_test_accuracy,mean_test_precision,mean_test_recall,mean_test_f1
3,0.061356,0.005612,0.006937,0.000764,0.900888,0.891014,0.863754,0.875425


#### Regressão Logística

In [ ]:
fit_grid_search(lr_, gridsearch_param_lr, 'accuracy', X, y)

Melhores parâmetros: {'C': 1}
Melhor score (accuracy): 0.910669


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,mean_test_accuracy,mean_test_precision,mean_test_recall,mean_test_f1
2,0.123258,0.09902,0.010029,0.004006,0.910669,0.893977,0.881954,0.887178


#### Random Forest

In [ ]:
fit_grid_search(rf_, gridsearch_param_rf, 'accuracy', X, y)

Melhores parâmetros: {'max_depth': None, 'n_estimators': 100}
Melhor score (accuracy): 0.929577


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,mean_test_accuracy,mean_test_precision,mean_test_recall,mean_test_f1
2,0.773095,0.007136,0.047647,0.014469,0.929577,0.919703,0.908986,0.912758


#### SVM

In [ ]:
fit_grid_search(svm_, gridsearch_param_svm, 'accuracy', X, y)

Melhores parâmetros: {'C': 10, 'gamma': 0.01}
Melhor score (accuracy): 0.834821


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,mean_test_accuracy,mean_test_precision,mean_test_recall,mean_test_f1
6,1.060252,0.232801,0.761939,0.253872,0.834821,0.777217,0.818515,0.796781
